In [1]:
import torch
import csv
import os
import random
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, random_split
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import optuna
from optuna.trial import TrialState



from dataset_reader import Traces_Dataset
from mlp_model import MLP

In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
target_features = 7
DIR = os.getcwd()
EPOCHS = 1
BATCH_SIZE = 1024

# N_TRAIN_EXAMPLES = 200 * 30
# N_VALID_EXAMPLES = 200 * 10

In [3]:
def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 5)
    layers = []

    in_features = 321
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 256, 1024)
        layers.append(nn.Linear(in_features, out_features))
        
        if trial.suggest_categorical(f"use_batchnorm_l{i}", [True, False]):
            layers.append(nn.BatchNorm1d(out_features))  # Assuming 1D input
            
        layers.append(nn.ReLU())
        # p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        # layers.append(nn.Dropout(p))
        in_features = out_features

    layers.append(nn.Linear(in_features, target_features))
    # layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)

In [8]:
def get_dataset(trial):
# def get_dataset():
    # batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128, 256, 512, 1024])
    
    dataset = Traces_Dataset('dataset_test.csv')
    dataset.split_dataset(0.95, 0.05, 0)
    dataset.clean_features()
    dataset.find_mean_std()
    dataset.normalize()
    # print(dataset.inputs.shape)

    # initialize train, val, test set
    X_train = dataset[dataset.train_set.indices][0]
    Y_train = dataset[dataset.train_set.indices][1]

    X_val = dataset[dataset.val_set.indices][0]
    Y_val = dataset[dataset.val_set.indices][1]

    X_test = dataset[dataset.test_set.indices][0]
    Y_test = dataset[dataset.test_set.indices][1]

    # initialize dataloader 
    train_dataset = TensorDataset(X_train, Y_train)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

    val_dataset = TensorDataset(X_val, Y_val)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

    return train_loader, val_loader

In [9]:
# train_loader, valid_loader = get_dataset()

In [10]:
def objective(trial):
    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    # optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    # lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    # optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    # weight_decay = trial.suggest_loguniform('weight_decay', 1e-5, 1e-2)
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
    
    

    # Get the FashionMNIST dataset.
    train_loader, valid_loader = get_dataset(trial)

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (train_inputs, train_targets) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            # if batch_idx * 10 >= N_TRAIN_EXAMPLES:
            #     break

            # data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
            train_inputs, train_targets = train_inputs.to(DEVICE), train_targets.to(DEVICE)

            optimizer.zero_grad()
            train_outputs = model(train_inputs)
            loss = nn.MSELoss()(train_outputs, train_targets)
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        total_val_loss = 0.0
        with torch.no_grad():
            for batch_idx, (val_inputs, val_targets) in enumerate(valid_loader):
                # Limiting validation data.
                # if batch_idx * 10 >= N_VALID_EXAMPLES:
                #     break
                
                # data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                val_inputs, val_targets = val_inputs.to(DEVICE), val_targets.to(DEVICE)

                val_outputs = model(val_inputs)
                # Get the index of the max log-probability.
                val_loss = nn.MSELoss()(val_outputs, val_targets)
                total_val_loss += val_loss.item()

            # Average validation loss for the epoch
        avg_val_loss = total_val_loss / len(valid_loader)

        trial.report(avg_val_loss, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return avg_val_loss

In [11]:
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=100, timeout=None, n_jobs=-1, show_progress_bar=True)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2024-04-01 21:43:10,437] A new study created in memory with name: no-name-47f487ac-62f1-4142-bb50-5ed46ea04b2e


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2024-04-01 21:43:10,979] Trial 1 finished with value: 3233.842041015625 and parameters: {'n_layers': 1, 'n_units_l0': 574, 'use_batchnorm_l0': True}. Best is trial 1 with value: 3233.842041015625.
[I 2024-04-01 21:43:11,014] Trial 9 finished with value: 3190.1943359375 and parameters: {'n_layers': 2, 'n_units_l0': 764, 'use_batchnorm_l0': True, 'n_units_l1': 517, 'use_batchnorm_l1': False}. Best is trial 9 with value: 3190.1943359375.
[I 2024-04-01 21:43:11,037] Trial 2 finished with value: 3232.742919921875 and parameters: {'n_layers': 2, 'n_units_l0': 517, 'use_batchnorm_l0': False, 'n_units_l1': 807, 'use_batchnorm_l1': True}. Best is trial 9 with value: 3190.1943359375.
[I 2024-04-01 21:43:11,067] Trial 8 finished with value: 3228.986328125 and parameters: {'n_layers': 2, 'n_units_l0': 949, 'use_batchnorm_l0': True, 'n_units_l1': 286, 'use_batchnorm_l1': True}. Best is trial 9 with value: 3190.1943359375.
[I 2024-04-01 21:43:11,106] Trial 3 finished with value: 3241.67919921875 

In [ ]:
optuna.importance.get_param_importances(study)